In [1]:
import boto3
import numpy as np
import io
import pickle
import pymysql
import sys
import os

In [2]:
# Let's use Amazon S3
s3 = boto3.resource('s3')

In [3]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

amazon-braket-c5c33f4595cd
cf-templates-1eqlum4iklsii-us-east-2
lukinsivdatabucket


## Option 1: Upload and download pickled data

In [4]:
# Upload some data
s3_client = boto3.client('s3')
data = np.zeros(10)

In [5]:
# upload without using disk
# from https://stackoverflow.com/questions/48049557/how-to-write-npy-file-to-s3-directly
my_array_data = io.BytesIO()
pickle.dump(data, my_array_data)
my_array_data.seek(0)
s3_client.upload_fileobj(my_array_data, 'lukinsivdatabucket', 'your-file.pkl')


In [6]:
# download data
# download without using disk
my_array_data2 = io.BytesIO()
s3_client.download_fileobj('lukinsivdatabucket', 'your-file.pkl', my_array_data2)
my_array_data2.seek(0)
my_array2 = pickle.load(my_array_data2)

# check that everything is correct
np.allclose(data, my_array2)

True

## Option 2: Upload and download file

In [7]:
#alternativly: save file

data_txt = np.savetxt("testdata.txt", data)
data_open = open("testdata.txt", 'rb')


In [8]:
s3.Bucket('lukinsivdatabucket').put_object(Key='testdata.txt', Body=data_open)

s3.Object(bucket_name='lukinsivdatabucket', key='testdata.txt')

In [9]:
# retrieve it by manually downloading it
s3_client.download_file('lukinsivdatabucket', 'testdata.txt', 'downloaded_testdata.txt')

## Connecting to RDS

In [11]:
from decouple import config
from sqlalchemy import create_engine
from sqlalchemy import URL

from sqlalchemy_utils import create_database, database_exists


In [2]:
use_aws = False

In [3]:
# Retrieve value from .env file

HOST= config("HOST")
PORT= config("PORT")
DBNAME= config("DBNAME")
USER= config("USER")
PASSWORD= config("PASSWORD")

url_object = URL.create(
    "mysql+pymysql",
    username=USER,
    password=PASSWORD,  # plain (unescaped) text
    host=HOST,
    database=DBNAME,
)



In [17]:
if use_aws:
    engine = create_engine(url_object)
else:
    engine = create_engine('sqlite:///pylabnet.sql', echo=True)


    if not database_exists(engine.url):
        create_database(engine.url)

    print(database_exists(engine.url))

True


In [14]:
connection = engine.connect()

In [6]:
# setup database

from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String, DateTime
from sqlalchemy.sql import func

from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class Chip(Base):
    __tablename__ = "chip"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    created_by: Mapped[str] = mapped_column(String(30))

    time_created = mapped_column(DateTime(timezone=True), server_default=func.now())
    time_updated = mapped_column(DateTime(timezone=True), onupdate=func.now())

    devices: Mapped[List["Device"]] = relationship(
        back_populates="chip", cascade="all"
    )

    def __repr__(self) -> str:
        return f"Chip(id={self.id!r}, name={self.name!r})"

class Device(Base):
    __tablename__ = "device"

    id: Mapped[int] = mapped_column(primary_key=True)
    chip_id: Mapped[int] = mapped_column(ForeignKey("chip.id"))
    name: Mapped[str] = mapped_column(String(30))


    chip: Mapped["Chip"] = relationship(back_populates="devices")

    experiments: Mapped[List["Experiment"]] = relationship(
            back_populates="device", cascade="all"
        )

    sivs: Mapped[List["SiV"]] = relationship(
            back_populates="devices", cascade="all"
        )


    def __repr__(self) -> str:
        return f"Device(id={self.id!r}, name={self.name!r})"

class SiV(Base):
    __tablename__ = "siv"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    device_id: Mapped[int] = mapped_column(ForeignKey("device.id"))


    devices: Mapped[List["Device"]] = relationship(
            back_populates="sivs", cascade="all"
        )

    experiments: Mapped[List["Experiment"]] = relationship(
            back_populates="siv", cascade="all"
        )

    def __repr__(self) -> str:
        return f"Device(id={self.id!r}, name={self.name!r})"


class Experiment(Base):
    __tablename__ = "experiment"

    id: Mapped[int] = mapped_column(primary_key=True)
    siv_id: Mapped[int] = mapped_column(ForeignKey("siv.id"), nullable=True)
    device_id: Mapped[int] = mapped_column(ForeignKey("device.id"), nullable=True)

    name: Mapped[str] = mapped_column(String(30))


    siv: Mapped["SiV"] = relationship(back_populates="experiments")
    device: Mapped["Device"] = relationship(back_populates="experiments")


    def __repr__(self) -> str:
        return f"Experiment(id={self.id!r}, name={self.name!r})"

In [7]:
Base.metadata.create_all(engine)


2023-02-17 15:37:08,623 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-17 15:37:08,624 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("chip")
2023-02-17 15:37:08,625 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-17 15:37:08,626 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("chip")
2023-02-17 15:37:08,626 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-17 15:37:08,627 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("device")
2023-02-17 15:37:08,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-17 15:37:08,627 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("device")
2023-02-17 15:37:08,628 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-17 15:37:08,628 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("siv")
2023-02-17 15:37:08,629 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-17 15:37:08,629 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("siv")
2023-02-17 15:37:08,630 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-17 15:3

In [8]:
from sqlalchemy.orm import Session
from sqlalchemy import select


with Session(engine) as session:

    chip_1 = Chip(
        name="20_09_B",
        devices=[Device(name="A_a_c")],
    )
   
    session.add_all([chip_1])

    session.commit()


In [16]:
from sqlalchemy import select





stmt = select(Device).where(Device.name == "A_a_c")
target_device = session.scalars(stmt).one()

In [17]:
target_device

Device(id=1, name='A_a_c')

In [21]:
with Session(engine) as session:

    odmr = Experiment(
        name="ODMR"
    )

    laserscan = Experiment(
        name="LASERSCAN"
    )
   
    session.add_all([odmr, laserscan])

    session.commit()

In [24]:
stmt = select(Experiment).where(Experiment.name == "ODMR")
experiment = session.scalars(stmt).first()


In [25]:
target_device.experiments = [experiment]

In [26]:
session.commit()

/tmp/ipykernel_17188/631814250.py:1: SAWarning: Object of type <Device> not in session, add operation along 'Experiment.device' won't proceed
  session.commit()


In [28]:
target_device.experiments

[Experiment(id=1, name='ODMR')]